In [1]:
import os
import pandas as pd

In [2]:
os.listdir()

['agency.txt',
 '.ipynb_checkpoints',
 'trips.txt',
 'google_transit_dublinbus.zip',
 'shapes.txt',
 'stop_times.txt',
 'routes.txt',
 'calendar.txt',
 'calendar_dates.txt',
 'Untitled.ipynb',
 'transfers.txt',
 'stops.txt']

In [5]:
stops_df = pd.read_csv('stops.txt')
stops_df.columns

Index(['stop_id', 'stop_name', 'stop_lat', 'stop_lon'], dtype='object')

In [14]:
stops_dict = {}

In [15]:
for row in stops_df.itertuples():
    stops_dict[row[1]]={'lat':row[3],'lon':row[4]}
    
    

In [16]:
stops_dict

{'8220B007612': {'lat': 53.341346779490898, 'lon': -6.2505294803674509},
 '8220DB000002': {'lat': 53.352244361138297, 'lon': -6.2637232189188703},
 '8220DB000003': {'lat': 53.352308551432507, 'lon': -6.2638107421682498},
 '8220DB000004': {'lat': 53.352574513184997, 'lon': -6.2641754860379697},
 '8220DB000006': {'lat': 53.352749335704601, 'lon': -6.2644538039643392},
 '8220DB000007': {'lat': 53.352840910578394, 'lon': -6.2645702612174903},
 '8220DB000008': {'lat': 53.353272271482197, 'lon': -6.2651836765739999},
 '8220DB000010': {'lat': 53.353392103647302, 'lon': -6.2653891238995394},
 '8220DB000011': {'lat': 53.356958303001797, 'lon': -6.2646283106692806},
 '8220DB000012': {'lat': 53.356804940544997, 'lon': -6.2645894713639105},
 '8220DB000014': {'lat': 53.358531237875795, 'lon': -6.2627765057087101},
 '8220DB000015': {'lat': 53.360088584135895, 'lon': -6.261090470313599},
 '8220DB000016': {'lat': 53.366391557141107, 'lon': -6.3056141835273705},
 '8220DB000017': {'lat': 53.362917073275

In [54]:
stops_map = {key:{} for key in stops_dict}

In [17]:
os.listdir()

['agency.txt',
 '.ipynb_checkpoints',
 'trips.txt',
 'google_transit_dublinbus.zip',
 'shapes.txt',
 'stop_times.txt',
 'routes.txt',
 'calendar.txt',
 'calendar_dates.txt',
 'transfers.txt',
 'stops.txt',
 'Mapping stops shapes distances etc.ipynb']

In [21]:
stops = pd.read_csv('stop_times.txt')

In [22]:
trips = pd.read_csv('trips.txt')

In [23]:
stops_trips = pd.merge(stops,trips,on=['trip_id'])

In [24]:
shapes = pd.read_csv('shapes.txt')

In [25]:
stops_trips.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,route_id,service_id,shape_id,trip_headsign,direction_id
0,3643.y102m.60-1-d12-1.1.O,15:40:00,15:40:00,8240DB000226,1,Sandymount,0,0,0.000000,60-1-d12-1,y102m,60-1-d12-1.1.O,Shanard Road (Shanard Avenue) - Saint John's R...,0
1,3643.y102m.60-1-d12-1.1.O,15:40:47,15:40:47,8240DB000228,2,Sandymount,0,0,261.136188,60-1-d12-1,y102m,60-1-d12-1.1.O,Shanard Road (Shanard Avenue) - Saint John's R...,0
2,3643.y102m.60-1-d12-1.1.O,15:41:26,15:41:26,8240DB000229,3,Sandymount,0,0,484.925289,60-1-d12-1,y102m,60-1-d12-1.1.O,Shanard Road (Shanard Avenue) - Saint John's R...,0
3,3643.y102m.60-1-d12-1.1.O,15:42:30,15:42:30,8240DB000227,4,Sandymount,0,0,836.995679,60-1-d12-1,y102m,60-1-d12-1.1.O,Shanard Road (Shanard Avenue) - Saint John's R...,0
4,3643.y102m.60-1-d12-1.1.O,15:43:09,15:43:09,8240DB000230,5,Sandymount,0,0,1066.461783,60-1-d12-1,y102m,60-1-d12-1.1.O,Shanard Road (Shanard Avenue) - Saint John's R...,0


In [26]:
for trip_id in stops_trips['trip_id'].unique():
    df = stops_trips[stops_trips['trip_id']==trip_id]
    if df['shape_id'].nunique()>1:
        print('ooops')

KeyboardInterrupt: 

In [55]:
visited = 0
for shape_id in stops_trips['shape_id'].unique():
    mined_patterns = set()
    gf=stops_trips[stops_trips['shape_id']==shape_id]
    for trip_id in gf['trip_id'].unique():
        df  = gf[gf['trip_id']==trip_id]
        if tuple(stop for stop in df['stop_id']) not in mined_patterns:
            print(trip_id)
            
            mined_patterns.add(tuple(stop for stop in df['stop_id']))
            shape = pd.DataFrame(shapes[shapes['shape_id']==shape_id])
            trip = pd.DataFrame(df)
            max_distance = shape['shape_dist_traveled'].max()
            current_distance = 0
            stop_pos = 0
            shape_pos = 0
            current_stop = trip.iloc[stop_pos]['stop_id']
            current_coords = []
            while current_distance < max_distance:
                
                if trip.iloc[stop_pos + 1]['shape_dist_traveled'] <\
                                                shape.iloc[shape_pos+1]['shape_dist_traveled']:
                    
                    stops_map[current_stop][trip.iloc[stop_pos+1]['stop_id']] = current_coords
                    current_distance = trip.iloc[stop_pos+1]['shape_dist_travelled']
                    stop_pos += 1
                    current_coords = []
                    visited +=1
                    current_stop = trip.iloc[stop_pos]['stop_id']
                elif trip.iloc[stop_pos + 1]['shape_dist_traveled'] >\
                                                    shape.iloc[shape_pos+1]['shape_dist_traveled']:
                        
                    shape_pos+=1
                    current_coords.append({'lat':shape.iloc[shape_pos]['shape_pt_lat'],\
                                          'lon':shape.iloc[shape_pos]['shape_pt_lon']})
                    current_distance = shape.iloc[shape_pos]['shape_dist_traveled']
                    
                else:
                    stops_map[current_stop][trip.iloc[stop_pos+1]['stop_id']] = current_coords
                    current_distance = trip.iloc[stop_pos+1]['shape_dist_traveled']
                    current_coords.append({'lat':shape.iloc[shape_pos]['shape_pt_lat'],\
                                          'lon':shape.iloc[shape_pos]['shape_pt_lon']})
                    shape_pos+=1
                    stop_pos += 1
                    current_coords = []
                    visited+=1
                    current_stop = trip.iloc[stop_pos]['stop_id']
                    
                
                
                
                
                
                
            
    
    

3643.y102m.60-1-d12-1.1.O
3667.y102m.60-1-d12-1.2.O
3720.y102m.60-1-d12-1.3.O
3676.y102m.60-1-d12-1.4.I
3634.y102m.60-1-d12-1.5.I
213.y102m.60-102-d12-1.78.O
212.y102m.60-102-d12-1.79.I
7731.y102o.60-102-d12-1.80.I
2210.y102m.60-104-d12-1.81.O
2211.y102m.60-104-d12-1.82.I
1885.y102m.60-11-d12-1.15.O
1955.y102m.60-11-d12-1.16.O
1887.y102m.60-11-d12-1.17.I
1943.y102m.60-11-d12-1.18.I
3248.y102m.60-111-d12-1.83.O
3187.y102m.60-111-d12-1.84.I
11404.y102o.60-111-d12-1.85.I
11412.y102o.60-111-d12-1.86.I
669.y102m.60-114-d12-1.87.O
670.y102m.60-114-d12-1.88.I
2420.y102m.60-116-d12-1.89.O
1316.y102m.60-116-d12-1.90.I
5924.y102m.60-118-d12-1.91.I
450.y102m.60-120-d12-1.92.O
531.y102m.60-120-d12-1.93.O
451.y102m.60-120-d12-1.94.I
6224.y102m.60-120-d12-1.95.I
13709.y102n.60-122-d12-1.96.O
8078.y102o.60-122-d12-1.97.O
13708.y102n.60-122-d12-1.98.I
8033.y102o.60-122-d12-1.99.I
6378.y102m.60-122-d12-1.100.O
6425.y102m.60-122-d12-1.101.O
6369.y102m.60-122-d12-1.102.I
6375.y102m.60-122-d12-1.103.I
750

16110.y102n.60-40-d12-1.294.O
3300.y102m.60-40-d12-1.295.I
3324.y102m.60-40-d12-1.296.I
3260.y102m.60-40-d12-1.297.I
3400.y102m.60-40B-d12-1.298.O
3315.y102m.60-40B-d12-1.299.O
16224.y102n.60-40B-d12-1.300.I
11618.y102o.60-40D-d12-1.301.O
3332.y102m.60-40D-d12-1.302.O
11610.y102o.60-40D-d12-1.303.I
3329.y102m.60-40D-d12-1.304.I
3395.y102m.60-40D-d12-1.305.O
3405.y102m.60-40D-d12-1.306.I
3.y102m.60-41-d12-1.307.O
37.y102m.60-41-d12-1.308.O
9.y102m.60-41-d12-1.309.O
45.y102m.60-41-d12-1.310.O
4.y102m.60-41-d12-1.311.I
6.y102m.60-41-d12-1.312.I
199.y102m.60-41-d12-1.313.I
13410.y102n.60-41-d12-1.314.I
7681.y102o.60-41-d12-1.315.O
9425.y102o.60-41-d12-1.316.O
44.y102m.60-41A-d12-1.317.I
47.y102m.60-41B-d12-1.318.O
7641.y102o.60-41B-d12-1.319.O
48.y102m.60-41B-d12-1.320.I
5.y102m.60-41C-d12-1.321.O
60.y102m.60-41C-d12-1.322.I
11.y102m.60-41C-d12-1.323.I
5828.y102m.60-41X-d12-1.324.O
5823.y102m.60-41X-d12-1.325.O
5827.y102m.60-41X-d12-1.326.O
5824.y102m.60-41X-d12-1.327.I
5822.y102m.60-41X-d

In [27]:
print(stops_trips['shape_id'].unique())

['60-1-d12-1.1.O' '60-1-d12-1.2.O' '60-1-d12-1.3.O' '60-1-d12-1.4.I'
 '60-1-d12-1.5.I' '60-102-d12-1.78.O' '60-102-d12-1.79.I'
 '60-102-d12-1.80.I' '60-104-d12-1.81.O' '60-104-d12-1.82.I'
 '60-11-d12-1.15.O' '60-11-d12-1.16.O' '60-11-d12-1.17.I'
 '60-11-d12-1.18.I' '60-111-d12-1.83.O' '60-111-d12-1.84.I'
 '60-111-d12-1.85.I' '60-111-d12-1.86.I' '60-114-d12-1.87.O'
 '60-114-d12-1.88.I' '60-116-d12-1.89.O' '60-116-d12-1.90.I'
 '60-118-d12-1.91.I' '60-120-d12-1.92.O' '60-120-d12-1.93.O'
 '60-120-d12-1.94.I' '60-120-d12-1.95.I' '60-122-d12-1.96.O'
 '60-122-d12-1.97.O' '60-122-d12-1.98.I' '60-122-d12-1.99.I'
 '60-122-d12-1.100.O' '60-122-d12-1.101.O' '60-122-d12-1.102.I'
 '60-122-d12-1.103.I' '60-123-d12-1.104.O' '60-123-d12-1.105.O'
 '60-123-d12-1.106.I' '60-123-d12-1.107.I' '60-13-d12-1.19.O'
 '60-13-d12-1.20.O' '60-13-d12-1.21.O' '60-13-d12-1.22.O'
 '60-13-d12-1.23.O' '60-13-d12-1.24.O' '60-13-d12-1.25.O'
 '60-13-d12-1.26.O' '60-13-d12-1.27.I' '60-13-d12-1.28.I'
 '60-13-d12-1.29.I' '60-1

In [42]:
import pickle

In [63]:
with open('trimmed_stops_shapes_map.pickle','wb') as handle:
    pickle.dump(trimmed_stops_map,handle, protocol=pickle.HIGHEST_PROTOCOL)

In [56]:
stops_map

{'8220B007612': {'8220DB002811': [{'lat': 53.341347219681502,
    'lon': -6.2505594988491611},
   {'lat': 53.341160562855904, 'lon': -6.2507023042126395},
   {'lat': 53.340700281523702, 'lon': -6.2511867049329011},
   {'lat': 53.340700281523702, 'lon': -6.2511867049329011}]},
 '8220DB000002': {'8220DB000192': [{'lat': 53.352305711325208,
    'lon': -6.2636155706244194},
   {'lat': 53.353224082995602, 'lon': -6.2649602978257688},
   {'lat': 53.353877575396794, 'lon': -6.2660153811310604},
   {'lat': 53.354364092066497, 'lon': -6.2673326584603295},
   {'lat': 53.354540869164104, 'lon': -6.267746127077169},
   {'lat': 53.354758854005901, 'lon': -6.2679025384032689},
   {'lat': 53.354946355098107, 'lon': -6.2684357622843203},
   {'lat': 53.355020178424098, 'lon': -6.2685679765416911},
   {'lat': 53.355599703026293, 'lon': -6.2682440185491801},
   {'lat': 53.356028262530792, 'lon': -6.2680463610065997},
   {'lat': 53.356162350753095, 'lon': -6.2679958531309996},
   {'lat': 53.35616235075309

In [57]:
print(visited)
print(len(stops_map))

24125
4691


In [58]:
total = 0
empty = 0
for stop in stops_map:
    
    total +=1
    if len(stops_map[stop])==0:
        empty+=1
        
        
print(total)
print(empty)

4691
59


In [51]:
a={}
print(len(a))

0


In [ ]:
trip_id = stops_trips['trip_id'].unique()[0]
trip = stops_trips[stops_trips['trip_id']==trip_id]
shape = shapes[shapes['shape_id']==trip['shape_id'][0]]

In [62]:
trimmed_stops_map={}
for stop in stops_map:
    try:
        trimmed_stop = str(int(stop[-4:]))
    except:
        trimmed_stop = str(stop)
    trimmed_stops_map[trimmed_stop]={}
    for stopb in stops_map[stop]:
        try:
            trimmed_stopb = str(int(stopb[-4:]))
        except:
            trimmed_stopb=str(stopb)
            
        trimmed_stops_map[trimmed_stop][trimmed_stopb]=stops_map[stop][stopb]
    

In [64]:
trimmed_stops_map

{'7612': {'2811': [{'lat': 53.341347219681502, 'lon': -6.2505594988491611},
   {'lat': 53.341160562855904, 'lon': -6.2507023042126395},
   {'lat': 53.340700281523702, 'lon': -6.2511867049329011},
   {'lat': 53.340700281523702, 'lon': -6.2511867049329011}]},
 '2': {'192': [{'lat': 53.352305711325208, 'lon': -6.2636155706244194},
   {'lat': 53.353224082995602, 'lon': -6.2649602978257688},
   {'lat': 53.353877575396794, 'lon': -6.2660153811310604},
   {'lat': 53.354364092066497, 'lon': -6.2673326584603295},
   {'lat': 53.354540869164104, 'lon': -6.267746127077169},
   {'lat': 53.354758854005901, 'lon': -6.2679025384032689},
   {'lat': 53.354946355098107, 'lon': -6.2684357622843203},
   {'lat': 53.355020178424098, 'lon': -6.2685679765416911},
   {'lat': 53.355599703026293, 'lon': -6.2682440185491801},
   {'lat': 53.356028262530792, 'lon': -6.2680463610065997},
   {'lat': 53.356162350753095, 'lon': -6.2679958531309996},
   {'lat': 53.356162350753095, 'lon': -6.2679958531309996}],
  '4726': 